In [1]:
####################
## Initial stuff
####################

# Load functions, modules and global variables required in our pipeline
from simulate_structures_functions import *
from importlib import reload  
from htmd.builder.charmm import _recoverProtonations

# Our main path
basepath = basepath = os.getcwd()+'/'
# Other Paths
strucpath = basepath + 'data_structures/'
resultspath = basepath + 'simulation_output/'
membranepdb = basepath + 'membrane/popc36_box_renumbered.pdb'
topparpath = basepath + 'toppar/TOP_PARAMS_ACE3/'#toppar= topology+parameters
ligandsdict_path = basepath + 'ligands.json'
modres_path = basepath + 'modified_residues.json'
slurmpath = basepath+'fake_slurm/'
# Path to slurm queing system binaries
# In our case, Ismael designed a bunch of small bash scripts (fake_slurm) which do ssh to Hydra and execute slurm there
path= os.environ['PATH']

# Modify path to include fake slurm
%env PATH=$path:$slurmpath

# USER INPUTS: introudce a json dictionary with the data for your system in the folowing format
"""
[
    {
        "name" : "NAME",
        "pdbfile" : "PATH/TO/YOUR_PDB.pdb",
        "modres" : ["MODIFIED_RESNAME1", "MODIFIED_RESNAME"],
        "ligands" : [{
                        "resname":"SMALMOL_RESNAME1",
                        "name" : "SMALMOL_NAME1",
                        "covalently_bound" : false # If smalmol is covalently bount to peptide
                    }],
        "pdbcode" : "PDBCODE",
        "curated" : true, # If system has been already properly protonated
        "sod2x50" : true # If system requires addition of sodium near 2x50
    },
"""
input_dict = json_dict(basepath+"inputs.json")
    
# Load topology, parameter and stream files with our current basepath
topos = [os.path.join(topparpath,file) for file in toposfilenames] 
params = [os.path.join(topparpath,file) for file in paramsfilenames]
streams = [os.path.join(topparpath,file) for file in streamsfilenames]


/home/david/miniconda3/lib/python3.6/site-packages/htmd/versionwarnings.py:28: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/david/miniconda3/lib/python3.6/site-packages/htmd/versionwarnings.py:34: UserWarning: As of HTMD 1.21 support for ACEMD v2 has stopped. Please use ACEMD3 instead as well as the corresponding equilibration and production protocols. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.21');`
  , UserWarning)
2021-11-30 09:48:16,591 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2021-11-30 09:48:17,213 - matplotlib.font_manager - INFO - Failed to extract font properties from /usr/share/fonts/truetype/noto/NotoColor


Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/

New stable HTMD version (1.26.5 python[3.9,<3.10.0a0,3.6,<3.7.0a0,3.8,<3.9.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.22.1).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.26.5 htmd=1.26.5 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



ModuleNotFoundError: No module named 'htmd.protocols.equilibration_v3'

In [ ]:
########################################
# Part 1: Prepare parameters for ligands
########################################

# Save mol2 files of ligand molecules and modres present in systems
(modresdict,ligandsdict,pdbfilesdict) = save_smalmol_mol2(input_dict)

# Get topology-parameter files for ligands
get_lig_toppar(ligandsdict, basepath, username, password, pdbfiles = pdbfilesdict)

# Get topology-parameter files for modified residues
get_modres_toppar(modresdict, basepath, username, password, pdbfiles = pdbfilesdict)

In [59]:
###########################
## Part 2: Build the models 
###########################

# Iterate by GPCRdb structures to simulate
for entry in input_dict:    
#    try:

        # Entry's data
        name = entry['name']
        pdbcode = entry['pdbcode']
        pdbfile = basepath+entry['pdbfile']
        curated = entry['curated']
        sod = "sod_yes" if entry['sod2x50'] else "sod_no"
        
        for apo in [False]:
            
            #Starting simulation
            start_time = time.time()        
            sysname = name+'_apo' if apo else name
            mystrucpath = strucpath+sysname+'/'
            os.makedirs(mystrucpath, exist_ok=True)

            # Skip if there is already a model build for this
            if os.path.exists(resultspath+'build/'+sysname+'/structure.pdb'):
                print('Build model for '+sysname+' already exists. Skipping...')
                continue

            # Check if simulation is aminergic
            aminergic = gpcrdb_dict[pdbcode]['family'].startswith('001_001')
            adenosine = gpcrdb_dict[pdbcode]['family'].startswith('001_006_001')
            
            ## Load structure
            #(sod2x50, watered_filename) = internal_waters(mystrucpath, pdbcode, gpcrdb_dict, apo, pdbfile,sod=sod)
            #gpcrdb_mol = Molecule(watered_filename)
            sod2x50 =True
            gpcrdb_mol = Molecule(pdbfile)
                
            # If the pipeline is running in 'apoform mode', remove any non-protein, non-ion, non-water thing on the system      
            # Delete also sod2x50 (we don't want them here)
            if apo:
                gpcrdb_mol.remove('not (protein or water or ion) or element Na')
                
            # Remove unnecessary ligand molecules: mostly crystalization detergents, quelants, buffers,
            # or post-traductional glicosilations
            gpcrdb_mol.remove('resname '+' '.join(blacklist))
            
            # Remove 2x50Sodium from non-A-class GPCRs
            if not gpcrdb_dict[pdbcode]['family'].startswith('001'):
                gpcrdb_mol.remove('element NA')

            # Get aligned OPM structure
            thickness,opm_mol = get_opm(pdbcode)

            # Ismael's function to add labels (segid) for 'ligand' and 'protein' parts of the system
            gpcrdb_mol_fixed,prot_segids = fix_and_prepare_input(gpcrdb_mol,name,modresdict,new_pdb_chain)
            
            # If there's any, parameterize and rename covalent-bound ligands
            if not apo:
                (gpcrdb_mol_fixed, covligs) = covalent_ligands(gpcrdb_mol_fixed, name, ligandsdict)
            else:
                covligs = []
            
            # write file to remember to which chain in the original structure belongs each segment
            segchain_json(gpcrdb_mol_fixed, sysname, basepath, prot_segids)
            
            # Align structrues using sequences, and take first one
            alignment_results = sequenceStructureAlignment(gpcrdb_mol_fixed, opm_mol, maxalignments = 1)
            mol_aligned = alignment_results[0] 
            
            #Center to receptor XY
            center = np.mean(mol_aligned.get('coords',sel='chain P'),axis=0)
            mol_aligned.moveBy([-center[0],-center[1],0])
            
            # Prepare protein: asign titration states, flipping side chains of HIS, ASN and GLN; rotate some sidechains, optimize waters, etc.
            # Most of this is done with a HTMD function called proteinPrepare()
            # Skip step if we are working with curators structures
            prepared_mol = mol_aligned if curated else prepare_system(mol_aligned, pdbcode, thickness, sod2x50, aminergic, adenosine)
            
            #Add membrane
            print('Adding membrane...')
            membranemol = Molecule(membranepdb)
            mol_membraned, membrane_resnames, membrane_segids, xreps, yreps = add_membrane(prepared_mol, membranemol,prot_segids,membrane_distance)
            
            # Needed later for equilibration
            with open(mystrucpath+"const_sel.txt",'w') as out: 
                const_sel = 'segid '+' '.join(prot_segids)+' and name C CA N O or not (segid ' + \
                  ' '.join(prot_segids)+' or lipid or water or ions ) and noh or segid ION WAT and noh'
                out.write(const_sel)

            #Solvate
            print('Solvating...')
            mol_solvated = solvate_pdbmol(mol_membraned,membrane_segids,water_thickness,water_margin,buffer=buffer,coldist=coldist,prefix='WT')

            # Check if system has lone-pair hallogen atoms. If it does, use legacy CGenFF parameters
            has_halo = bool(len(gpcrdb_mol_fixed.get('name', 'element Br Cl I and not ion')))
            if has_halo:
                cgenff_par = [topparpath+'David_top_params/parameters/legacy_par_all36_cgenff.prm']
                cgenff_top = [topparpath+'David_top_params/topologies/legacy_top_all36_cgenff.rtf']
            else: 
                cgenff_par = [topparpath+'General_top_params/parameters/par_all36_cgenff.prm']
                cgenff_top = [topparpath+'General_top_params/topologies/top_all36_cgenff.rtf']
            
            #Obtain extra parameters for ligands and modified residues 
            ligstreams=extra_parameters(name, ligandsdict, modresdict, blacklist, covligs, basepath, has_halo)
            
            # Assignign terminology for cap atoms of protein chain, depending if it is the receptor protein or not
            caps = get_caps(prot_segids, mol_solvated)
            #{'P0': ['first ACE', 'last CT3'], 'P1': ['first ACE', 'last CT3']}

            #Pre-build model
            print('Pre-build...')
            prebuildmol = charmm.build(mol_solvated, 
                                       topo=topos+cgenff_top, 
                                       param=params+cgenff_par,
                                       stream=streams+ligstreams,
                                       caps=caps,
                                       outdir=resultspath+'/pre-build/'+sysname,
                                       ionize=False)

            # Save prebuild model topologies in files, and  store prebuild model in molecule object
            prebuild_psffile = prebuildmol.topoloc
            prebuild_pdbfile = os.path.splitext(prebuildmol.topoloc)[0]+'.pdb'
            prebuildmol = Molecule(prebuild_pdbfile)
            _recoverProtonations(prebuildmol)

            # Checking of aromatic insertions (takes quite a lot fo time)
            print('Checking aromatic insertions...')
            mol_removed,removed_indexes = remove_aromatic_insertions(prebuildmol,prot_segids, outpdb=resultspath+'/pre-build/'+sysname+'/aromatic_check.pdb')

            # Checking of water/lipid ratio
            lipid_num = len(set(prebuildmol.get('resid',sel='segid '+membrane_lipid_segid)))
            solv_num = len(mol_removed.get('index',sel='resname TIP3 and name OH2'))
            if float(solv_num) / lipid_num < 35:
                raise ValueError('Water/lipid ratio lower than 35.')

            #Renumber residues
            print('Renumbering...')
            mol_renumbered = renumber_resid_vmd(mol_removed,'segid '+' '.join(membrane_segids),by=2)

            # Ionizing system
            print('Ionizing...')
            molbuilt = charmm.build(mol_removed,
                                    topo=topos+cgenff_top, 
                                    param=params+cgenff_par,
                                    stream=streams+ligstreams,                        
                                    outdir=resultspath+'/ionize/'+sysname,
                                    saltconc=0.15,
                                    caps=caps)
            build_psffile = molbuilt.topoloc
            build_pdbfile = os.path.splitext(molbuilt.topoloc)[0]+'.pdb'
            molbuilt = Molecule(build_pdbfile)
            _recoverProtonations(molbuilt)

            #Building system
            print('Building...')
            molbuilt = renumber_resid_vmd(molbuilt,'segid "WT.*" or segid I',by=2)
            molbuilt = charmm.build(molbuilt, 
                                    topo=topos+cgenff_top, 
                                    param=params+cgenff_par,
                                    stream=streams+ligstreams,                        
                                    outdir=resultspath+'/build/'+sysname,
                                    caps=caps,ionize=False)

            print('End of %s after %s seconds\n' % (sysname, time.time() - start_time))

#    except Exception as e:
 #       print("model "+sysname+" could not be build because ",e)

2021-11-29 15:55:38,517 - moleculekit.molecule - INFO - Removed 0 atoms. 4785 atoms remaining in the molecule.
2021-11-29 15:55:41,375 - moleculekit.molecule - INFO - Removed 0 atoms. 4785 atoms remaining in the molecule.
2021-11-29 15:55:41,465 - moleculekit.molecule - INFO - Removed 3 atoms. 4782 atoms remaining in the molecule.
2021-11-29 15:55:41,713 - moleculekit.molecule - INFO - Removed 0 atoms. 4782 atoms remaining in the molecule.
2021-11-29 15:55:42,725 - moleculekit.tools.autosegment - INFO - Created segment P0 between resid 1 and 393.
2021-11-29 15:55:43,621 - moleculekit.molecule - INFO - Removed 0 atoms. 4782 atoms remaining in the molecule.
2021-11-29 15:55:44,114 - moleculekit.tools.sequencestructuralalignment - INFO - No segment was specified by the user for `mol`. Alignment will be done on all protein segments.
2021-11-29 15:55:44,222 - moleculekit.tools.sequencestructuralalignment - INFO - No segment was specified by the user for `ref` and multiple segments (['0', '1

Adding membrane...


2021-11-29 15:55:46,764 - htmd.builder.builder - INFO - Replicating Membrane 3x3
Replicating Membrane: 100%|██████████| 9/9 [00:05<00:00,  1.54it/s]
2021-11-29 15:58:15,578 - moleculekit.molecule - INFO - Removed 2189 atoms. 42835 atoms remaining in the molecule.
2021-11-29 15:58:20,718 - moleculekit.molecule - INFO - Removed 126 residues from appended Molecule due to collisions.


Solvating...
wataerbox Max and min:  [52.285786 53.262016 28.060892] [-47.308212 -46.377983 -27.734108]


2021-11-29 15:58:22,498 - htmd.builder.solvate - INFO - Using water pdb file at: /soft/system/easybuild/software/Miniconda3/4.7.10/lib/python3.6/site-packages/htmd/share/solvate/wat.pdb
2021-11-29 15:58:24,619 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2
Solvating: 100%|██████████| 8/8 [00:22<00:00,  2.84s/it]
2021-11-29 15:58:51,718 - htmd.builder.solvate - INFO - 12369 water molecules were added to the system.
2021-11-29 15:59:06,410 - moleculekit.molecule - INFO - Removed 582 atoms. 79048 atoms remaining in the molecule.


...

Building...


2021-11-29 16:42:39,020 - htmd.builder.charmm - INFO - Writing out segments.
2021-11-29 16:42:52,861 - htmd.builder.builder - INFO - 2 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 349, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 353, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 148, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYS', chain: 'P', resid: 227, insertion: '', segid: 'P0'>



2021-11-29 16:42:53,947 - htmd.builder.charmm - INFO - Starting the build.
2021-11-29 16:42:55,691 - htmd.builder.charmm - WARNING - Failed to set coordinates for 1 atoms.
2021-11-29 16:42:55,693 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 1 atoms.
2021-11-29 16:42:55,694 - htmd.builder.charmm - WARNING - Please check /gpcr/users/daranda/doctorat/PSYBIAS_simulations/simulation_output/build/otava37/log.txt for further information.
2021-11-29 16:42:55,694 - htmd.builder.charmm - INFO - Finished building.
2021-11-29 16:43:48,783 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (SER 285 CA P0 P) (SER 285 C P0 P) (ILE 315 N P0 P) (ILE 315 CA P0 P) " with indexes [3475, 3482, 3484, 3486]


End of otava37 after 730.3223831653595 seconds



In [6]:
#########################
## Part 3: Equillibration
#########################

# Iterate by GPCRdb structures to simulate
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    for apo in [False]:
        # Entry's data
        try:
            modelname = name+'_apo' if apo else name
            equildir = resultspath+'equil/'+modelname+'/'
            if os.path.exists(equildir+'output.xtc') or os.path.exists(equildir+'simrunning'):
                print(" structure %s already has been equilibrated" % modelname)
                continue

            if not os.path.exists(equildir):
                os.makedirs(equildir)

            # Taking vmd selection line
            const_sels = ""
            with open(strucpath+modelname+'/const_sel.txt', 'r') as outfile:
                const_sel += outfile.readlines()

            md = define_equilibration(const_sel)
            md.write(resultspath+'build/'+modelname,equildir)

            #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
            with open(equildir + 'run.sh', 'w') as f:
                f.write('#!/bin/bash\n%s > %slog.txt 2>&1' % (acemd_path, equildir))
        
            #Prepare slurm job  
            sq = SlurmQueue()
            sq.envvars = acemd_license
            sq.jobname = 'eql_'+modelname
            sq.datadir = None
            sq.partition = 'gpcr_gpu'
            sq.ngpu = 1
            sq.ncpu = 1
            sq.prerun = job_commands(equildir, '/home/daranda/'+modelname+'/')
            sq.memory = 2000
#             sq.exclude = ['arwen','aragorn','bifur']
            sq.nodelist = ['aragorn','arwen']
    
            # Submit
            sq.submit(equildir)
            
        except Exception as e:
            print("model "+modelname+" could not be send to equilibrate because of ",e)

 structure 5ZHP already has been equilibrated
 structure 5ZHP already has been equilibrated
 structure 6AKX already has been equilibrated
 structure 6AKX already has been equilibrated
 structure 5TE5 already has been equilibrated
 structure 5TE5 already has been equilibrated
 structure 6KQI already has been equilibrated
 structure 6KQI already has been equilibrated
model 6LI2 could not be send to equilibrate because of  name 'const_sel' is not defined
 structure 6LI2 already has been equilibrated
 structure 5UNH already has been equilibrated
 structure 5UNH already has been equilibrated
 structure 5XRA already has been equilibrated
 structure 5XRA already has been equilibrated
 structure 5T04 already has been equilibrated
 structure 5T04 already has been equilibrated
 structure 5OLO already has been equilibrated
 structure 5OLO already has been equilibrated
 structure 6LRY already has been equilibrated
 structure 6LRY already has been equilibrated
 structure 5X33 already has been equil

 structure 5UIW already has been equilibrated
 structure 6J21 already has been equilibrated
 structure 6J21 already has been equilibrated


In [9]:
#####################
## Part 4: Production
#####################

# Production protocol
md = define_production(timestep, trajperiod)

# For each structure 
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    for apo in [False]:
#     for apo in [True, False]:
        # must match with equildir in equilibration launcher code and contain input and output of equilibration.
        modelname = name+'_apo' if apo else name
        equildir = '%s/equil/%s/' % (resultspath, modelname)
        if not os.path.exists(equildir):
            print("structure %s has not been yet equillibrated. Skipping...")
            continue
        for rep in range(1,repnum+1):
            
            try: 
                # If simulation for this PDB has already been run
                proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
                if os.path.exists(proddir+'/output.xtc') or os.path.exists(proddir+'simrunning'):
                    print("replicate %d of structure %s already has been simulated" %(rep, modelname))
                    continue

                print('submitting replicate %d of %s' % (rep, modelname))
                # directory copy output of equilibration to production input (initial working directory for run_prod.sh).
                md.write(equildir,proddir)

                sq = SlurmQueue()
                sq.envvars = acemd_license
                sq.jobname = modelname+'_pr'+str(rep)
                sq.datadir = None
                sq.partition = 'gpcr_gpu'
                sq.prerun = job_commands(proddir, '/home/daranda/%s_pr_%d/'%(modelname,rep))
                sq.ngpu = 1
                sq.ncpu = 1
                sq.exclude = ['aragorn']

                #Substitute run.sh generated by HTMD by a different one, adapted to the specified path of ACEMD
                with open(proddir + 'run.sh', 'w') as f:
                    f.write('#!/bin/bash\n%s >%slog.txt 2>&1' % (acemd_path,proddir))

                sq.submit(proddir)
                
            except Exception as e:
                print("model "+modelname+" could not be send to equilibrate because of ",e)            


replicate 1 of structure 5ZHP already has been simulated
replicate 2 of structure 5ZHP already has been simulated
replicate 3 of structure 5ZHP already has been simulated
replicate 1 of structure 5ZHP already has been simulated
replicate 2 of structure 5ZHP already has been simulated
replicate 3 of structure 5ZHP already has been simulated
replicate 1 of structure 6AKX already has been simulated
replicate 2 of structure 6AKX already has been simulated
replicate 3 of structure 6AKX already has been simulated
replicate 1 of structure 6AKX already has been simulated
replicate 2 of structure 6AKX already has been simulated
replicate 3 of structure 6AKX already has been simulated
replicate 1 of structure 5TE5 already has been simulated
replicate 2 of structure 5TE5 already has been simulated
replicate 3 of structure 5TE5 already has been simulated
replicate 1 of structure 5TE5 already has been simulated
replicate 2 of structure 5TE5 already has been simulated
replicate 3 of structure 5TE5 a

2021-10-26 12:44:18,656 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2021-10-26 12:44:19,224 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6KQI_apo/rep_1/


submitting replicate 2 of 6KQI


2021-10-26 12:44:48,803 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2021-10-26 12:44:49,071 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6KQI_apo/rep_2/


submitting replicate 3 of 6KQI


...

2021-10-26 14:56:38,696 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (GLY 230 CA P P) (GLY 230 C P P) (ASP 231 N P P) (ASP 231 CA P P) " with indexes [3367, 3370, 3372, 3374]
2021-10-26 14:56:39,942 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2021-10-26 14:56:40,669 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6J21/rep_1/


submitting replicate 2 of 6J21


2021-10-26 14:57:12,698 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (GLY 230 CA P P) (GLY 230 C P P) (ASP 231 N P P) (ASP 231 CA P P) " with indexes [3367, 3370, 3372, 3374]
2021-10-26 14:57:13,942 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2021-10-26 14:57:14,283 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6J21/rep_2/


submitting replicate 3 of 6J21


2021-10-26 14:57:47,697 - htmd.builder.builder - WARNING - Found cis peptide bond in 1 frames: [0] in the omega diheral "Angle of (GLY 230 CA P P) (GLY 230 C P P) (ASP 231 N P P) (ASP 231 CA P P) " with indexes [3367, 3370, 3372, 3374]
2021-10-26 14:57:48,946 - htmd.protocols.production_v6 - WARNING - Lipids detected in input structure. We highly recommend setting useconstantratio=True for membrane simulations.
2021-10-26 14:57:49,282 - jobqueues.slurmqueue - INFO - Queueing /gpcr/users/daranda/doctorat/GPCR_simulations/simulation_output/production/6J21/rep_3/


In [12]:

##########################
## Part 5: Wrap Structures
##########################

# Wrap trajectories obtained during production with an htmd command
gpcr_sel = "protein and chain "+new_pdb_chain
for entry in input_dict:    
    name = entry['name']
    pdbcode = entry['pdbcode']
    for apo in [False]:
        try:
            modelname = pdbcode+'_apo' if apo else pdbcode
            mymol_pdb = Molecule('%sproduction/%s/rep_1/structure.pdb' % (resultspath, modelname))
            for rep in range(1,repnum+1):
            #for rep in [3]:
                start_time = time.time()        
                print('wrapping replicate %d of %s' % (rep, modelname))
                proddir='%sproduction/%s/rep_%d/' % (resultspath, modelname, rep)
                rep = str(rep)

                # To avoid repeating wrapping in Trajectories already wrapped, check the existance of this file
                outname = proddir+'output_wrapped.xtc'
                if os.path.exists(outname):
                    print('replicate already wrapped. Skipping...')
                    continue
                
                # Wrap system
                mymol = Molecule('%sproduction/%s/rep_1/structure.psf' % (resultspath, modelname))
                mymol.read(proddir+'output.xtc')
                mymol.wrap(gpcr_sel)

                # Align frames and write
                mymol.align('all', refmol=mymol_pdb)
                mymol.write(outname)

            print('End of %s after %s seconds\n' % (modelname, time.time() - start_time))
        except Exception as e:
            print("model "+modelname+" could not be wrapped because of ",e)            




wrapping replicate 1 of 6MEO
replicate already wrapped. Skipping...
wrapping replicate 2 of 6MEO
replicate already wrapped. Skipping...
wrapping replicate 3 of 6MEO
replicate already wrapped. Skipping...
wrapping replicate 1 of 5TE5
replicate already wrapped. Skipping...
wrapping replicate 2 of 5TE5
replicate already wrapped. Skipping...
wrapping replicate 3 of 5TE5
replicate already wrapped. Skipping...
wrapping replicate 1 of 5WIU
replicate already wrapped. Skipping...
wrapping replicate 2 of 5WIU
replicate already wrapped. Skipping...
wrapping replicate 3 of 5WIU
replicate already wrapped. Skipping...
wrapping replicate 1 of 6LRY


/home/david/miniconda3/lib/python3.6/site-packages/moleculekit/align.py:16: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, A), array(float32, 2d, A))
  covariance = np.dot(P.T, Q)
/home/david/miniconda3/lib/python3.6/site-packages/moleculekit/align.py:54: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float32, 2d, C), array(float32, 2d, A))
  all1 = np.dot(all1, rot.T)


End of 6LRY after 8.202301025390625 seconds

wrapping replicate 2 of 6LRY
End of 6LRY after 4.026684045791626 seconds

wrapping replicate 3 of 6LRY
End of 6LRY after 3.4290406703948975 seconds

wrapping replicate 1 of 4EJ4
replicate already wrapped. Skipping...
wrapping replicate 2 of 4EJ4
replicate already wrapped. Skipping...
wrapping replicate 3 of 4EJ4
replicate already wrapped. Skipping...


In [7]:
###################################
## Part 5: Upload results to GPCRmd
###################################

#### Functions
def resp_to_dict(resp):
    # Convert a json reponse into a dictionary
    return eval(resp.content.decode('UTF-8').replace('true', 'True').replace('false','False'))

def get_headers(s, subm_id):
    """
    Obtain headers, csrftoken and sessionid from current session object
    """
    sessionid = str(s.cookies['sessionid'])
    csrftoken = str(s.cookies['csrftoken'])
    headers = {
        'Referer' : mainurl+'/dynadb/step1/'+subm_id,
        'Cookie' : 'csrftoken=%s; sessionid=%s' %(csrftoken,sessionid),
        'X-CSRFToken' : csrftoken,
    }
    return(sessionid, csrftoken, headers)


def check_chains(pdbcode, mymol):
    """
    Check how many chains from the original PDB structure remain in mymol structure
    And to which Segments of our structure they correspond
    """
    # Load blosum score matrix to align proteins
    blosum62 = substitution_matrices.load("BLOSUM62")

    # Obtain sequences for original PDB file chains, and classifying them by chainID
    # Also check which segment corresponds to what chain
    pdbmol = Molecule(pdbcode)
    chainseg = {}
    chainset = set(pdbmol.get('chain', sel='protein'))
    pdbmol_segseqs = pdbmol.sequence()
    pdbmol_chainseqs = {}
    for chain in chainset:
        segid = np.unique(pdbmol.get('segid', sel='chain '+chain))[0]
        pdbmol_chainseqs[chain] = pdbmol_segseqs[segid]

    # Merging all protein chains in our systems into a single megachain
    mymol_megachain = ''
    for seg,chain in mymol.sequence().items():
        mymol_megachain = mymol_megachain + chain

    # Aligning sequences from original PDB to simulated PDB megasequence
    # To know which chains from the original PDB are preserved
    chain_present = {}
    segtochain = {}
    # For each chain in the original PDB file
    for chain,seq in pdbmol_chainseqs.items():
        chain_present[chain] = False
        # For each segment in our molecule
        for seg,myseq in mymol.sequence().items():
            mylen = len(myseq)
            # Align our molecule segments to the pdb chains
            aligs = pairwise2.align.localms(seq, myseq, 5,-1, -1.5, -1)
            # Check if any of the alignments has more than 4 score by position
            is_present = any([ (alig.score/mylen) > 4 for alig in aligs ]) 
            if is_present:
                chain_present[chain] = is_present
                segtochain[seg] = chain

    return (chain_present, segtochain)

def get_pdb_info(pdbcode, mymol, ligandsdict):
    """
    Get informarion of the system specified in the pdbcode from the RCSB-PDB webpage.
    Mainly uniprot sequences, chainIDs and uniprot codes for the chains
    """
    
    # Make string for selecting lignads in rcsb's api
    ligs = '"'+'","'.join(ligandsdict[pdbcode].keys())+'"'
    
    #Check which chains from the original PDB structure are preserved in mymol structure
    (chain_present, segtochain) = check_chains(pdbcode, mymol)
    
    # Get ligand molecules present in our system (basically anything that is not a protein)
    ligset = set(mymol.get('resname', sel='not protein'))
    
    # Extract information from pdb webpage using api
    ligdict = dict()
    protdict = dict()
    datadict = dict()
    
    # Get information from PDB api
    pdbdict = requests.get('https://data.rcsb.org/graphql?query={\
        entry(entry_id: "'+pdbcode+'") {\
            exptl {method}\
            polymer_entities {\
                entity_poly{pdbx_strand_id}\
                rcsb_polymer_entity{pdbx_description}\
                rcsb_polymer_entity_align{\
                    aligned_regions {length}\
                    reference_database_accession \
                    reference_database_isoform \
                }\
            }\
        }\
        chem_comps(comp_ids: ['+ligs+']) {\
            rcsb_chem_comp_descriptor {InChIKey}\
            chem_comp{id,name}\
        }\
    }').json()['data']
    
    # Get ligand information and classify it (in case the system actually has ligands)
    if len(pdbdict['chem_comps'])>0:
        for lig in pdbdict['chem_comps']:
            ligandInchi = lig['rcsb_chem_comp_descriptor']['InChIKey']
            ligandResname = lig['chem_comp']['id']
            ligandName = lig['chem_comp']['name']
    
            # Exclude ligands not present in our simulated system
            if ligandResname in ligset:
                ligdict[ligandResname] = (ligandName,ligandInchi)

    # Extract protein chains information
    for poly in pdbdict['entry']['polymer_entities']:
    
        # Get alignment information in order to determine which uniprot segment we should take
        prevallen = 0
        uniprotkbac = ''
        isoform = ''
        for alreg in poly['rcsb_polymer_entity_align']:
            allen =  sum([seg['length'] for seg in alreg['aligned_regions']])
            unicode = alreg['reference_database_accession']
            myisoform = alreg['reference_database_isoform']
            if allen > prevallen:
                uniprotkbac = unicode
                isoform = myisoform
            else:
                uniprotkbac = uniprotkbac
                isoform = isoform
            prevallen = allen            
            
        uniname = poly['rcsb_polymer_entity']['pdbx_description'].lower()
        chainIds = poly['entity_poly']['pdbx_strand_id'].split(',')

        # Determine if this polymer (chain) is a GPCR
        isgpcr = (('receptor' in uniname) or ('rhodopsin' in uniname)) 

        # If this chain is present in our mymol structure
        for chainId in chainIds:
            if chain_present[chainId]:
                # Get which segment(s) this chain is assigned to. SKip if none
                segs = [ seg for seg,chain in segtochain.items() if chain == chainId ]
                if not segs:
                    continue
                # Order data for this chain
                protdict[chainId] = {
                    'uniprotkbac' : uniprotkbac,
                    'isoform' : isoform if isoform else '1',
                    'isgpcr' : isgpcr,
                    'segs' : segs,
                }

    # Determine experimental method used, and use the corresponding id in GPCRmd database
    method = pdbdict['entry']['exptl'][0]['method'].lower()
    if 'x-ray' in method: 
        method_id = 0
    elif 'nmr' in method:
        method_id = 1
    elif 'electron microscopy' == method:
        method_id = 4
    else:
        method_id = 5 # Other method
            
    return (protdict,ligdict,segtochain,method_id)

def login(s):
    # Login into GPCRmd 
    # Will you go, lassie, go? And we'll all gooo toghetheeeer
    headers = {
        'Cookie': 'csrftoken=cuGA6CSGmXfMbLwqlPoGjLLN7QkO6rZ7',
        'Referer': mainurl+'/accounts/login/',
    }
    datalogin = {
        'username': 'david',
        'password': 'Ameboid',
        'next' : '/accounts/memberpage/',
        'csrfmiddlewaretoken' : 'cuGA6CSGmXfMbLwqlPoGjLLN7QkO6rZ7'
    }
    print('loging into GPCRmd')
    logo = s.post(mainurl+'/accounts/login/', 
               data=datalogin,
               headers=headers)
    return s

def new_submission(s, mainurl):
    """
    Create a new submission entry in GPCRmd
    """
    response_new = s.get(mainurl + '/dynadb/step0/')
    soup = BeautifulSoup(response_new.text, 'html.parser')
    step1_link = soup.find('form',attrs={'id' : 'mainform'}).get('action')
    subm_id = step1_link.split('/')[-2]
    print('new submission %s created'%subm_id)
    return subm_id

def check_completeness(prodpath):
    """
    Check if we actually have the three necessary replicates for submission
    """

    incomplete = False
    for trajid in range(1,4):
        trajpath = '%s/rep_%d/output_wrapped.xtc'%(prodpath,trajid)
        if not os.path.exists(trajpath):
            incomplete = True

    return(incomplete)

def new_step1(s, subm_id, pdbcode, trajperiod, timestep, prodpath, modelfile, method_id, apo):
    """
    Fullfill and send the step 1 of the new submissionform
    # Looong liiie the fieeeelds of Athenryyyyy...
    # Where once, we watch, the small seabirds flyyyyy....
    """
    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)
    
    # All the stuff we need to send 
    sysname = pdbcode+' apoform' if apo else pdbcode+' complex'
    data_submit = {
        'name' : sysname,
        'type' : 0 if apo else 1, # 0 for apo, 1 for complex
        'pdbid' : pdbcode,
        'description' : 'Autosubmission',
        'source_type' : method_id,
        'id_dynamics_methods': '1', #Molecular mechanics,
        'software': 'ACEMD3',
        'sversion': 'GPUGRID', # TODO: Is that correct??,
        'ff': 'CHARMM',
        'ffversion': '36m Feb 2016',
        'id_assay_types': 1, # Orthosteric (un)/binding,
        'id_dynamics_membrane_types': 2, # Homogeneus membrane,
        'id_dynamics_solvent_types': 2, # TIP3P solvent,
        'timestep': timestep,
        'delta': (trajperiod*timestep)/10e6,
        'add_info': 'autosubmission', # time/frames
    }
    files_submit = {
        'dynamics' : open(prodpath+'structure.pdb', 'rb'),
        'model' : open(modelfile, 'rb')
    }

    # Submit step 1
    rep = s.post(mainurl+'/dynadb/step1_submit/'+subm_id+'/',
            headers = headers,
            files = files_submit,
            data = data_submit)
    print('step1 finalized ',rep)

    
def smalmol_getdata(s, molid, inchikey, subm_id, moltype, resname, sdfpath=""):
    """
    Get data for small molecules 
    """
    
    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)

    # Get info of molecule
    rep = s.get(mainurl+'/dynadb/smalmol_info/?inchikey=%s&submission_id=%s'%(inchikey,subm_id),
        headers = headers,
        )
    rep_dict = rep.json()
    
    # Prepare submission dictionary with the data of this molecule
    mol_datasubmit = {
        "smalmol_type"+molid : moltype,
        "smalmol_inchikey"+molid : inchikey,
        "smalmol_name"+molid : rep_dict['name'],
        "smalmol_resname"+molid : resname,
        "smalmol_iupac"+molid : rep_dict['iupac'],
        "smalmol_chemblid"+molid :  rep_dict['chemblid'],
        "smalmol_cid"+molid :  rep_dict['cid'],
        "smalmol_inchi"+molid : rep_dict['inchi'],
        "smalmol_sinchi"+molid :  rep_dict['sinchi'],
        "smalmol_sinchikey"+molid :  rep_dict['sinchikey'],
        "smalmol_smiles"+molid :  rep_dict['smiles'],
        "smalmol_netcharge"+molid :  rep_dict['net_charge'],
        "smalmol_synonims"+molid :  rep_dict['other_names'],
        "smalmol_description"+molid :  '',
        "image_path"+molid :  rep_dict['imagepath']
    }
    
    # Add SDFfile to the submission if required
    mol_files = {} if rep_dict['inGPCRmd'] else {"sdfmol"+molid : open(sdfpath)} 
    
    return(mol_datasubmit,mol_files)

def new_step2(s, subm_id, ligdict):
    """
    Fullfil and send step2 of new submission form
    """    
    print('initiating step 2: small molecule data')

    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)
    
    # Introduce common small molecules (waters, lipids and ions)
    i = 0
    num_entries = []
    step2_data = {
        'csrfmiddlewaretoken' : csrftoken,
        'submit' : 'submit'
    }
    step2_files = {}
    common_mols = {
        'TIP3' : ('XLYOFNOQVPJJNP-UHFFFAOYSA-N',6),
        'POPC' : ('WTJKGGKOPKCXLL-VYOBOKEXSA-N',7),
        'SOD' : ('FKNQFGJONOIPTF-UHFFFAOYSA-N',8),
        'CLA' : ('VEXZGXHMUGYJMC-UHFFFAOYSA-M',8)
    }
    for resname,(inchikey,moltype) in common_mols.items():
        (mol_datasubmit, mol_files) = smalmol_getdata(s, str(i), inchikey, subm_id, moltype, resname)
        step2_data.update(mol_datasubmit)
        step2_files.update(mol_files)
        num_entries.append(i)
        i+=1

    # Introduce ligands: all will be defined as orthosteric ligands
    lignames = []
    for lig,(name,inchikey) in ligdict.items():
        # Skip this if molecule is apoform
        if apo:
            break
        # Avoid blacklisted molecules or ions
        if (lig in detergent_blacklist) or (lig in glucids_blacklist)  or (len(lig) == 2):
            continue
        # Download ligand, store it into temporary file
        response = requests.get('https://files.rcsb.org/ligands/view/'+lig+'_ideal.sdf')
        with open('tmpfile.sdf','wb') as tmpout:
            tmpout.write(response.content)

        # CLR (cholesterol) is usually a experimental lipid 
        moltype = '3' if lig=='CLR' else '0'
            
        # Retrieve molecule information
        (mol_datasubmit, mol_files) = smalmol_getdata(s, str(i), inchikey, subm_id, moltype, lig, 'tmpfile.sdf')
        step2_data.update(mol_datasubmit)
        step2_files.update(mol_files)
        num_entries.append(i)
        i+=1
        os.remove('tmpfile.sdf')
    
    # Add number of entries
    step2_data['num_entries'] = ','.join(map(str, num_entries))
    
    # Replace empty entries with an empty string
    for key,val in step2_data.items():
        if not val:
            step2_data[key] = ''
    
    # Submit step2
    rep = s.post(mainurl+'/dynadb/step2_submit/'+subm_id+'/',
        headers = headers,
        data = step2_data,
        files = step2_files)
    print('step2 finalized ',rep)
    
def new_step3(s, subm_id, protdict):
    """
    Fullfil and sent the step3 (protein chains) of the new submission form
    """
    print('initiating step 3: protein chains')

    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)

    # Retrieve segment information using GPCRmd submission form
    entrydict = s.get(mainurl+'/dynadb/find_prots/'+subm_id+'/',
        headers = headers).json()
    
    segments = {seg['segid'] : seg for entry in entrydict for seg in entry['segments']}

    # Start dict with the data to be send into GPCRmd
    # But thougth they paved the footways here with goldust, I still would chose... my Isle of Innesfree
    step3_data = {
        'csrfmiddlewaretoken':csrftoken,
        'submission_id' : subm_id,
        'submit' : 'Submit',
    }

    # Iterate over pdb chains. For each one we'll create an entry
    i=0
    entrynum = []
    for chain,chain_data in protdict.items():
        ii = str(i)
        uniprotkbac = chain_data['uniprotkbac']
        isgpcr = chain_data['isgpcr']
        isoform = chain_data['isoform']
        chainsegs = chain_data['segs']

        # Retrieve info using uniprot code
        unidict = s.get(mainurl+'/dynadb/prot_info/'+subm_id+'/',
                          params = {
                              'uniprotkbac' : uniprotkbac,
                              'isoform' : isoform,
                              'submission_id' : subm_id,
                          },
                          headers = headers
                         ).json()

        # We'll obtain the alignment of our protein sequence and its UNIPROT sequence using
        # GPCRmd's tool for do it
        data_alig = {
            'unisequence' : unidict['Sequence'],
            'submission_id' : subm_id
        }

        # Get segments for alignment data
        j = 0
        segnums = []
        for seg in chainsegs:
            seginfo = segments[seg]
            ss = str(j)
            data_alig['chain'+ss]= seginfo['chain']
            data_alig['segid'+ss]= seg
            data_alig['from'+ss]= seginfo['resid_from']
            data_alig['to'+ss]= seginfo['resid_to']
            segnums.append(ss)

            # Store segment information as well in the submit dict
            step3_segdata = {
                ss+'seg'+ii: j,
                ss+'pdbid'+ii: seginfo['pdbid'],
                ss+'sourcetype'+ii: seginfo['source_type'],
                ss+'chain'+ii: seginfo['chain'],
                ss+'segid'+ii: seg,
                ss+'from_resid'+ii: seginfo['resid_from'],
                ss+'to_resid'+ii: seginfo['resid_to'],
            }
            step3_data.update(step3_segdata)

            j += 1
        data_alig['segnums']=','.join(segnums)
        step3_data['num_segs'+ii]=','.join(segnums)

        # Send alignment request
        alignment = s.post(mainurl+'/dynadb/get_alignment/',
                      data = data_alig,
                      headers = headers
                     ).text

        # Get mutations
        mutations_dict = s.post(mainurl+'/dynadb/protein/get_mutations/',
                      data = {
                          'alignment' : alignment,
                          'sequence' : unidict['Sequence'],
                      },
                      headers = headers
                     ).json()['mutations']

        # Store mutations in submission dictionary
        h = 0
        mutnums = []
        for mut in mutations_dict:
            hh = str(h)
            step3_mutdata = {
                hh+'from'+ii : mut['from'],
                hh+'to'+ii : mut['to'],
                hh+'resid'+ii : mut['resid'],
            }
            step3_data.update(step3_mutdata)
            mutnums.append(hh)
            h+=1
        step3_data['num_muts'+ii]=','.join(mutnums)

        # Get chain name according to uniprot
        names_array = unidict['Protein names'].split(" (")
        names_array = list(map(lambda x: re.sub("\)$", "", x), names_array)) # Convert the name string into name aray
        for a in names_array:
            print('A name', a)
        name = names_array.pop() # We get the first name as the official one
        other_names = ';'.join(names_array)
        print('the name ', name)
        print('formated other names', other_names)
        
        # Start storing information to send in step3
        step3_entrydata = {
            'isoform'+ii : isoform,
            'prot_uniprot'+ii : uniprotkbac,
            'name'+ii : name,
            'aliases'+ii : other_names,
            'species_code'+ii : unidict['Organism ID'],
            'species_name'+ii : unidict['Organism'],
            'unisequence'+ii : unidict['Sequence'],
        }
        step3_data.update(step3_entrydata)

        entrynum.append(ii)    
        i+=1

    # Store number of entries
    step3_data['num_entries'] = ','.join(entrynum)

    # Submit step3
    rep = s.post(mainurl+'/dynadb/step3_submit/'+subm_id+'/',
        headers = headers,
        data = step3_data)
    
    print('step3 finalized ',rep)


def new_step4(s, subm_id, prodpath, repath):
    """
    Fullfil and sent the step4 (files) of the new submission form
    """
    print('initiating step 4: simulation files')

    # Get headers and stuff
    (sessionid, csrftoken, headers)=get_headers(s, subm_id)

    # Prepare trajectories
    step4_files = []
    for trajid in range(1,4):
#         trajpath = '%s/rep_%d/output.xtc'%(prodpath,trajid)
#         step4_files.append(('trj', ('traj%d.xtc'%trajid, open(trajpath, 'rb'))))
        trajpath = '%s/rep_%d/trajtest.dcd'%(prodpath,trajid)
        step4_files.append(('trj', ('trajtest.dcd', open(trajpath, 'rb'))))
    
    # Prepare the rest of files (parameters files will name-changed to avoid problems with filextensions)
    step4_files.append(
        ('dyn', ('structure.psf', open(repath+'structure.psf','rb')))
    )
    step4_files.append(
        ('prm', ('parameters.prm', open(repath+'parameters','rb')))
    )
        
    # Encode data
    m=MultipartEncoder(fields=step4_files)
    headers['Content-Type'] = m.content_type
    
    # Submit step4
    rep = s.post(mainurl+'/dynadb/step4_submit/'+subm_id+'/',
        headers = headers,
        data = m)
    
    print('step4 finalized ',rep)



In [8]:
#### Actual Simulation submission
# mainurl = 'http://localhost:8000' 
mainurl = 'https://submission.gpcrmd.org'

# For each of the currently-working-with systems defined in Part 1
#for pdbcode in pdb_set:
#for pdbcode in testset:
for pdbcode in ['5NDZ']:
    for apo in [False]:
#     for apo in [True, False]:
        
        try:
            # Set the path to the folder containing this simulation's files
            sysname = pdbcode+"_apo" if apo else pdbcode+"_complex" 
            prodpath = resultspath+'production/'+pdbcode+'_apo/' if apo else resultspath+'production/'+pdbcode+'/' 
            repath = prodpath+'rep_1/'# For files no changing across representations
            modelfolder = "%sreceptor2curate_output/%s/"%(basepath, pdbcode) 
            modelpath = "%s%s_apo_curated.pdb"%(modelfolder,pdbcode) if apo else "%s%s_complex_curated.pdb"%(modelfolder,pdbcode)
            
            # Skip if not all trajectories    
            incomplete = check_completeness(prodpath)
            if incomplete:
                continue            
            
            # Load molecule
            mymol = Molecule(repath+'structure.pdb')
            print('Submitting '+sysname+' simulation...')

            ## Step -2: Get information of protein chains and ligand molecules from PDB web
            (protdict,ligdict,segtochain,method_id) = get_pdb_info(pdbcode, mymol, ligandsdict)

            ## Step -1: Login into GPCRmd
            with requests.Session() as s:
                login(s)

            ## Step 0: Create a new submission
            subm_id = new_submission(s, mainurl)

            ## Step 1: General information
            new_step1(s, subm_id, pdbcode, trajperiod, timestep, repath, modelpath, method_id, apo)

            ## Step 2: Small molecules 
            new_step2(s, subm_id, ligdict)   
    
            ## Step 3: Protein chains
            new_step3(s, subm_id, protdict)
            sdf

            ## Step 4: Dynamics information
            new_step4(s, subm_id, prodpath, repath)
        
        except Exception as e:
            print("Simulation "+sysname+" could not be submitted because ",e)            


2021-11-10 10:18:35,835 - moleculekit.readers - INFO - Attempting PDB query for 5NDZ


Submitting 5NDZ_complex simulation...
loging into GPCRmd
new submission 423 created
step1 finalized  <Response [200]>
initiating step 2: small molecule data
step2 finalized  <Response [200]>
initiating step 3: protein chains
A name Proteinase-activated receptor 2
A name PAR-2
A name Coagulation factor II receptor-like 1
A name G-protein coupled receptor 11
A name Thrombin receptor-like 1) [Cleaved into: Proteinase-activated receptor 2, alternate cleaved 1; Proteinase-activated receptor 2, alternate cleaved 2]
the name  Thrombin receptor-like 1) [Cleaved into: Proteinase-activated receptor 2, alternate cleaved 1; Proteinase-activated receptor 2, alternate cleaved 2]
formated other names Proteinase-activated receptor 2;PAR-2;Coagulation factor II receptor-like 1;G-protein coupled receptor 11
step3 finalized  <Response [200]>
Simulation 5NDZ_complex could not be submitted because  name 'sdf' is not defined
